# MSB1015 Scientific programming
## Heart failure predicition model


During this project the goal is predicting Heart failure in the kaggle challenge Heart Failure Prediction. The database contains 12 Clinical features and 1 target. The features consitute several different data types form interger numbers, precentages, and binarized data.


### Pre-Processing
The first step was loading in the data and a preliminary exploration of the data.

In [19]:
# Importing neccesary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading in the heart failure data

df = pd.read_excel (r'Tom_data.xlsx')

# 
print(df.dtypes)
print(list(df.columns))
print(df.head())

age                         float64
anaemia                       int64
creatinine_phosphokinase    float64
diabetes                      int64
ejection_fraction           float64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                float64
sex                           int64
smoking                       int64
time                          int64
DEATH_EVENT                   int64
dtype: object
['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT']
    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                     582.0         0               20.0   
1  55.0        0                    7861.0         0               38.0   
2  65.0        0                     146.0         0               20.0   
3  50.0        1         

The data consist of of 13 columns; Age, Anaemia, Creatine_phosphokinase, Diabetes, Ejection_fraction, High_blood_pressure, Platelets, Serum_creatine, Serum_sodium, Sex, smoking, Time, and Death event.

In [16]:
# changing columns (variables) to a logical data type
#   ejection fraction has a negative value and a impossible high value
#   serum_sodium has a impossible high value 

df["age"] = df["age"].apply(np.int64)
df = df.drop([149], axis=0)

lorem ispem